# Лабораторная работа №2. Лобанов Дмитрий ИУ5-25М
### Цель работы
Изучение продвинутых способов предварительной обработки данных для дальнейшего формирования моделей.
### Задание
1) Выбрать набор данных (датасет), содержащий категориальные и числовые признаки и пропуски в данных. Для выполнения следующих пунктов можно использовать несколько различных наборов данных (один для обработки пропусков, другой для категориальных признаков и т.д.) Просьба не использовать датасет, на котором данная задача решалась в лекции.
2) Для выбранного датасета (датасетов) на основе материалов лекций решить следующие задачи:
устранение пропусков в данных,
кодирование категориальных признаков,
нормализация числовых признаков.

## Ход работы

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "heart.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "johnsmith88/heart-disease-dataset",
  file_path,
)
df.shape

<ipython-input-1-fd806175d8e1>:12: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


(1025, 14)

In [ ]:
# Проверка структуры
print(df.info())
print("\nПропуски:\n", df.isnull().sum())
print("\nКатегориальные признаки:\n", df.select_dtypes(include='object').nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB
None

Пропуски:
 age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dty

Пропусков нет, поэтому добавим самостоятельно

In [ ]:
# Добавим искусственные пропуски — примерно 10% значений в некоторых колонках
np.random.seed(42)
for col in ['age', 'chol', 'sex', 'thal']:
    df.loc[df.sample(frac=0.1).index, col] = np.nan

# Проверим количество пропусков
print("Пропуски в данных:\n", df.isnull().sum())


Пропуски в данных:
 age         102
sex         102
cp            0
trestbps      0
chol        102
fbs           0
restecg       0
thalach       0
exang         0
oldpeak       0
slope         0
ca            0
thal        102
target        0
dtype: int64


In [ ]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52.0,1.0,0,125,212.0,0,1,168,0,1.0,2,2,3.0,0
1,53.0,1.0,0,140,203.0,1,0,155,1,3.1,0,0,3.0,0
2,70.0,1.0,0,145,174.0,0,1,125,1,2.6,0,0,3.0,0
3,NaN,1.0,0,148,203.0,0,1,161,0,0.0,2,1,3.0,0
4,62.0,0.0,0,138,294.0,1,1,106,0,1.9,1,3,2.0,0


## Обработка пропусков

In [ ]:
# Разделим признаки по типу
num_cols = ['age', 'chol']
cat_cols = ['sex', 'thal']

# Импьют числовых признаков (средним значением)
num_imputer = SimpleImputer(strategy='mean')
df[num_cols] = num_imputer.fit_transform(df[num_cols])

# Импьют категориальных признаков (модой)
cat_imputer = SimpleImputer(strategy='most_frequent')
df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])

print("Пропуски в данных:\n", df.isnull().sum())

Пропуски в данных:
 age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64


In [ ]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52.000000,1.0,0,125,212.0,0,1,168,0,1.0,2,2,3.0,0
1,53.000000,1.0,0,140,203.0,1,0,155,1,3.1,0,0,3.0,0
2,70.000000,1.0,0,145,174.0,0,1,125,1,2.6,0,0,3.0,0
3,54.307692,1.0,0,148,203.0,0,1,161,0,0.0,2,1,3.0,0
4,62.000000,0.0,0,138,294.0,1,1,106,0,1.9,1,3,2.0,0


## Кодирование категориальных признаков

In [ ]:
# One-Hot Encoding категориальных признаков
df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52.000000,1.0,0,125,212.0,0,1,168,0,1.0,2,2,3.0,0
1,53.000000,1.0,0,140,203.0,1,0,155,1,3.1,0,0,3.0,0
2,70.000000,1.0,0,145,174.0,0,1,125,1,2.6,0,0,3.0,0
3,54.307692,1.0,0,148,203.0,0,1,161,0,0.0,2,1,3.0,0
4,62.000000,0.0,0,138,294.0,1,1,106,0,1.9,1,3,2.0,0


## Нормализация числовых признаков

In [ ]:
scaler = StandardScaler()
df_encoded[num_cols] = scaler.fit_transform(df_encoded[num_cols])
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52.000000,1.0,0,125,212.0,0,1,168,0,1.0,2,2,3.0,0
1,53.000000,1.0,0,140,203.0,1,0,155,1,3.1,0,0,3.0,0
2,70.000000,1.0,0,145,174.0,0,1,125,1,2.6,0,0,3.0,0
3,54.307692,1.0,0,148,203.0,0,1,161,0,0.0,2,1,3.0,0
4,62.000000,0.0,0,138,294.0,1,1,106,0,1.9,1,3,2.0,0


## Вывод
В ходе работы данные были успешно подготовлены для анализа: пропуски заполнены средним и модой, категориальные признаки закодированы, а числовые нормализованы.